#miniFlow

A neural network is a graph of mathematical functions such as linear combinations and activation functions. The graph consists of **nodes**, and **edges**.

Nodes perform mathematical functions, The nodes and edges create a graph structure.

### Owen version of tensorflow

## MiniFlow Architecture

In [1]:
import numpy as np

In [ ]:
# Node defines the base set of properties that's every node holds.

class Node(object):
    def __init__(self):
        # Properites will go here!
        
        # A list of nodes with edges into this node
        self.inbound_nodes = inbound_nodes
        # the eventual value of this value
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        
        # New property, inputs to this node and their values are the 
        # partials of this node with respect to that input
        self.gradients = {}
        # set this node as an outbound node for all of this node's input
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            
        # A calculated value
        self.value = None
        
        def forward(self):
            """
            Forward propagation.
            Compute the output value based on 'inbound_nodes and
            store the result in self.value
            """
            
        def backward(self):
            """
            """
            

In [ ]:
# subclass of Nodes input just hold value

class Input(Node):
    def __init__(self):
        # An input node has no inbound nodes,
        Node.__init__(self)
        
        def forword(self, value=None):
            # Overwrite the value if none is passed in.
            if value is not None:
                self.value = value

In [ ]:
# Add Subclass

class Add(Node):
    def __init__(self, x, y):
        Node.__init(self, [x, y])
        
    def forword(self):
        """
        Set the value of this node('self.value') to the sum of inbound_nodes
        """
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value

In [ ]:
# Mul class

In [ ]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.
        """
        X = self.inbound_nodes[0].value
        Y = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b
    
    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

In [ ]:
# d_sigmoid : sigmoid*(1-sigmoid)
# d_cost/ d_sigmoid : grad_cost

class Sigmoid(Node):
    
    def __init__(self, node):
        Node.__init__(self, [node])
        
    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x)) # the `.` ensures that `1` is a float

    def forward(self):
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)
        
    def backward(self):
        """
        Calculates the gradient using the derivative of the sigmoid function
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            for n in self.outbound_nodes:
                grad_cost = n.gradients[self]
                sigmoid = self.value
                self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost

## SGD
a version of Gradient Descent where on each forward pass a batch of data is randomly sampled from total dataset, as emory constraints, SGD is an approximation of Gradient Descent.

SGD implementation involves:
1. Randomly sample a batch of data from the total dataset.
2. Running the network forward and backward to calculate the gradient (with data from (1)).
3. Apply the gradient descent update.
4. Repeat steps 1-3 until convergence or the loop is stopped by another mechanism (i.e. the number of epochs).

** x=x−α∗d_cos/d_x **


In [ ]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # Perform SGD
    #
    # Loop over the trainables
    for t in trainables:
        # Change the trainable's value by subtracting the learning rate
        # multiplied by the partial of the cost with respect this trainable.
        partial = t.gradidents[t]
        # partial of the cost (C) with respect to the trainable t is accessed
        t.value -= learning_rate * partial
        # value of the trainable is updated by d_x = x - learning_rate * detal_cost
        

In [ ]:
# cost function MSE

class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        m = self.inbound_nodes[0].value.shape[0]
        
        diff = y - a
        self.value = np.mean(diff**2)
        
    def backward(self):
        """
        Calculates the gradient of the cost.
        
        This is the final node of the network so outbound nodes are not a concern.
        """
        self.gradient[self.inbound_nodes[0]] - (2 / self.m) * self.diff
        self.gradient[self.inbound_nodes[1]] - (2 / self.m) * self.diff

In [2]:
# learning rate: empirically values in the range 0.1 to 0.0001 work well. 

# gradient descent: x = x - learning_rate * gradient_of_x
def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    # TODO: Implement gradient descent.
    
    # Return the new value for x
    x = x - learning_rate * gradx
    return x

In [ ]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [ ]:
# replace upper forward_pass 

def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted nodes.

    Arguments:
    
        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

In [ ]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

## two methods to run value through your graphs:
- topological_sort()
- forward_pass()

In [4]:
def f(x):
    """
    Quadratic function.

    It's easy to see the minimum value of the function
    is 5 when is x=0.
    """
    return x**2 + 5


def df(x):
    """
    Derivative of `f` with respect to `x`.
    """
    return 2*x


# Random number better 0 and 10,000. Feel free to set x whatever you like.
x = np.random.randint(0, 10000)
# TODO: Set the learning rate
learning_rate = 0.1
epochs = 100

for i in range(epochs+1):
    cost = f(x)
    gradx = df(x)
    print("EPOCH {}: Cost = {:.3f}, x = {:.3f}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, learning_rate)


EPOCH 0: Cost = 39463529.000, x = 12564.000
EPOCH 1: Cost = 25256660.360, x = 10051.200
EPOCH 2: Cost = 16164264.430, x = 8040.960
EPOCH 3: Cost = 10345131.035, x = 6432.768
EPOCH 4: Cost = 6620885.663, x = 5146.214
EPOCH 5: Cost = 4237368.624, x = 4116.972
EPOCH 6: Cost = 2711917.719, x = 3293.577
EPOCH 7: Cost = 1735629.140, x = 2634.862
EPOCH 8: Cost = 1110804.450, x = 2107.889
EPOCH 9: Cost = 710916.648, x = 1686.312
EPOCH 10: Cost = 454988.455, x = 1349.049
EPOCH 11: Cost = 291194.411, x = 1079.239
EPOCH 12: Cost = 186366.223, x = 863.392
EPOCH 13: Cost = 119276.183, x = 690.713
EPOCH 14: Cost = 76338.557, x = 552.571
EPOCH 15: Cost = 48858.476, x = 442.056
EPOCH 16: Cost = 31271.225, x = 353.645
EPOCH 17: Cost = 20015.384, x = 282.916
EPOCH 18: Cost = 12811.646, x = 226.333
EPOCH 19: Cost = 8201.253, x = 181.066
EPOCH 20: Cost = 5250.602, x = 144.853
EPOCH 21: Cost = 3362.185, x = 115.882
EPOCH 22: Cost = 2153.599, x = 92.706
EPOCH 23: Cost = 1380.103, x = 74.165
EPOCH 24: Cost =

##
So, each node will pass on the cost gradient to its inbound nodes and each node will get the cost gradient from it's outbound nodes. Then, for each node we'll need to calculate a gradient that's the cost gradient times the gradient of that node with respect to its inputs. Below I've written out this process for a Linear node.

    # Initialize a partial for each of the inbound_nodes.
    self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
    # Cycle through the outputs. The gradient will change depending
    # on each output, so the gradients are summed over all outputs.
    for n in self.outbound_nodes:
        # Get the partial of the cost with respect to this node.
        grad_cost = n.gradients[self]
        # Set the partial of the loss with respect to this node's inputs.
        self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
        # Set the partial of the loss with respect to this node's weights.
        self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
        # Set the partial of the loss with respect to this node's bias.
        self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

In [ ]:
## nn.py

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

In [ ]:
# use miniFlow nn to predict Boston house price

"""
Check out the new network architecture and dataset!

Notice that the weights and biases are
generated randomly.

No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample
# from miniflow import *  # i seperate this code in notebook

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 10
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


##

See traning [MNIST](http://yann.lecun.com/exdb/mnist/)